In [0]:
import pandas as pd

# Path to your extracted file
file_path = "/Volumes/workspace/default/miningaccidents/Accidents.txt"  # Replace with your local or cloud path

# Load the file using pipe delimiter and quoted values
df = pd.read_csv(
    file_path,
    sep="|",
    quotechar='"',
    encoding="latin1",  # Use 'latin1' encoding to handle decoding errors
    dtype=str  # Start with all fields as strings
)

# Strip whitespace from column names and convert to lowercase
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

# Convert relevant date fields
df['accident_dt'] = pd.to_datetime(df['accident_dt'], errors='coerce')
df['return_to_work_dt'] = pd.to_datetime(df['return_to_work_dt'], errors='coerce')

# Drop rows missing crucial information (e.g., mine_id or accident_dt)
df = df.dropna(subset=["mine_id", "accident_dt"])

# Preview cleaned data
print(df.head())


In [0]:
%python
# Analyze the top 10 locations from ug_location column
top_locations = df['ug_location_cd'].value_counts().head(10)

display(top_locations)

# fips_state_cd:object
# ug_location_cd:object


In [0]:
# Analyze the most common types of accidents
common_accidents = df['accident_type'].value_counts().head(10)

display(common_accidents)

In [0]:
# Gettng address data
#/Volumes/workspace/default/miningaccidents/AddressOfRecord.txt

import pandas as pd

# Path to your extracted file
file_path_add = "/Volumes/workspace/default/miningaccidents/AddressOfRecord.txt"  # Replace with your local or cloud path

# Load the file using pipe delimiter and quoted values
df_add = pd.read_csv(
    file_path_add,
    sep="|",
    quotechar='"',
    encoding="latin1",  # Use 'latin1' encoding to handle decoding errors
    dtype=str  # Start with all fields as strings
)

# Strip whitespace from column names and convert to lowercase
df_add.columns = [col.strip().lower().replace(" ", "_") for col in df_add.columns]

# print(df_add.head())
df_add.display()

In [0]:
%python
%pip install tabulate
import tabulate
import json
import pandas as pd

# Path to the downloaded file
file_path = '/Volumes/workspace/default/miningaccidents/lite.json'  # Update this if needed

# Step 1: Open and load JSON from the file
with open(file_path, 'r', encoding='utf-8') as f:
    stations = json.load(f)

# # Load the data again in case the previous state is not preserved
df = pd.read_json(file_path)

# Flatten the 'name' column
df_name = pd.json_normalize(df['name'], sep='_')
df = pd.concat([df, df_name], axis=1)

# Flatten the 'identifiers' column
df_identifiers = pd.json_normalize(df['identifiers'], sep='_')
df = pd.concat([df, df_identifiers], axis=1)

# Flatten the 'location' column
df_location = pd.json_normalize(df['location'], sep='_')
df = pd.concat([df, df_location], axis=1)

# Flatten the 'inventory' column
df_inventory = pd.json_normalize(df['inventory'], sep='_')
df = pd.concat([df, df_inventory], axis=1)

# Drop the original nested columns
df = df.drop(columns=['name', 'identifiers', 'location', 'inventory'])

# Display the first 5 rows of the flattened DataFrame
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
#print(df.info())
df.display()